# Convolutional Neural Networks卷积神经网络
## 卷积神经网络定义

卷积神经网络是人工神经网络的一种，已成为当前语音分析和图像识别领域的研究热点。它的权值共享网络结构使之更类似于生物神经网络，降低了网络模型的复杂度，减少了权值的数量。该优点在网络的输入是多维图像时表现的更为明显，使图像可以直接作为网络的输入，避免了传统识别算法中复杂的特征提取和数据重建过程。卷积网络是为识别二维形状而特殊设计的一个多层感知器，这种网络结构对平移、比例缩放、倾斜或者共他形式的变形具有高度不变性。

## 卷积神经网络的网络结构
![img](http://img.my.csdn.net/uploads/201304/10/1365562155_9356.jpg)
![img](http://img.my.csdn.net/uploads/201304/10/1365562217_2880.jpg)
卷积神经网络其实是一种特征卷积后的提取，将区域内图像的特征值处理提取
![img](http://img.my.csdn.net/uploads/201304/10/1365562245_8389.jpg)
![img](http://img.my.csdn.net/uploads/201304/10/1365562321_2400.jpg)
一般来说，CNN网络的前几层为卷积层和采样层（或者说池化层），在若干层卷积和池化以后，还有若干层全连接层（也就是传统神经网络），最后输出分类信息。大概的结构示意图如上图所示

### 卷积层函数 tf.nn.conv2d
def conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None,data_format=None, name=None):

前几个参数分别是input, filter, strides, padding, use_cudnn_on_gpu, …下面来一一解释 

input：待卷积的数据。格式要求为一个张量，[batch, in_height, in_width, in_channels].
分别表示 批次数，图像高度，宽度，输入通道数。 

filter： 卷积核。格式要求为[filter_height, filter_width, in_channels, out_channels].
分别表示 卷积核的高度，宽度，输入通道数，输出通道数。 

strides :一个长为4的list. 表示每次卷积以后卷积窗口在input中滑动的距离 

padding ：有SAME和VALID两种选项，表示是否要保留图像边上那一圈不完全卷积的部分。如果是SAME，则保留 

use_cudnn_on_gpu ：是否使用cudnn加速。默认是True


### 池化层函数 tf.nn.max_pool

def max_pool(value, ksize, strides, padding, data_format="NHWC", name=None):

value: 一个4D张量，格式为[batch, height, width, channels]，与conv2d中input格式一样

ksize: 长为4的list,表示池化窗口的尺寸 

strides: 池化窗口的滑动值，与conv2d中的一样 

padding: 与conv2d中用法一样。

In [2]:
# 导入数据
import tensorflow.examples.tutorials.mnist.input_data as input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [30]:
#定义变量和层
def weightVariable(shape):
    init = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(init)
 
def biasVariable(shape):
    init = tf.random_normal(shape)
    return tf.Variable(init)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
 
def maxPool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [33]:
#

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1]) #将输入按照 conv2d中input的格式来reshape，reshape

#定义第一层，卷积核是5x5,32通道，h_pool输出为?*14*14*32
W_conv1 = weightVariable([5,5,1,32])#输入通道是1，输出通道是32
B_conv1 = biasVariable([32])
h_conv1 = tf.nn.elu(conv2d(x_image,W_conv1) + B_conv1)
h_pool1 = maxPool(h_conv1)

#定义第二层，h_pool输出为?*7*7*64
W_conv2 = weightVariable([5,5,32,64])#输入通道是32，输出通道是64
B_conv2 = biasVariable([64])
h_conv2 = tf.nn.elu(conv2d(h_pool1, W_conv2) + B_conv2)
h_pool2 = maxPool(h_conv2)

#定义第三层，连接层
W_fc1 = weightVariable([7*7*64, 1024])
B_fc1 = biasVariable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.elu(tf.matmul(h_pool2_flat, W_fc1) + B_fc1)
keep_prob = tf.placeholder(tf.float32) # 这里使用了drop out,即随机安排一些cell输出值为0，可以防止过拟合
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#定义第四层，输出层
W_fc2 = weightVariable([1024,10])
B_fc2 = biasVariable([10])
h_fc2 = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + B_fc2)

y_ = tf.placeholder(tf.float32, [None, 10])

#定义loss函数和optimis函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(h_fc2), reduction_indices=[1]))# 损失函数，交叉熵
train_step = tf.train.AdadeltaOptimizer(1e-4).minimize(cross_entropy)# 使用adam优化
correct_prediction = tf.equal(tf.argmax(h_fc2, 1),tf.arg_max(y_,1))# 计算准确度
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#
sess = tf.InteractiveSession()
 init = tf.global_variables_initializer()
sess.run(init)
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%1000 == 0:
        # print(batch[1].shape)
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

sess.close()

C:\Users\YAN\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training accuracy 0.06
step 1000, training accuracy 0.08
step 2000, training accuracy 0.06
step 3000, training accuracy 0.12
step 4000, training accuracy 0.08
step 5000, training accuracy 0.04
step 6000, training accuracy 0.08
step 7000, training accuracy 0.1
step 8000, training accuracy 0.06
step 9000, training accuracy 0.12
step 10000, training accuracy 0.16
step 11000, training accuracy 0.04
step 12000, training accuracy 0.04
step 13000, training accuracy 0.14
step 14000, training accuracy 0.1
step 15000, training accuracy 0.14
step 16000, training accuracy 0.14


KeyboardInterrupt: 

In [29]:
sess.close()